In [11]:
import os
from os import listdir
import io
import re
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
# load train data and interpolate
train_data_df=pd.read_csv('../csv_data/train_patient.csv')
train_data_df = train_data_df.interpolate(method='linear').ffill().bfill() 
train_data_df.head(8)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_id,time
0,80.0,100.0,36.50,121.00,58.0,41.00,13.5,34.0,1.0,25.000000,...,160.00,77.27,1,0.0,1.0,-69.14,3,0,0,0
1,76.0,100.0,36.25,113.25,61.0,41.50,12.0,34.0,1.0,25.000000,...,161.80,77.27,1,0.0,1.0,-69.14,4,0,0,1
2,80.0,100.0,36.25,132.75,71.5,46.25,12.0,34.0,-1.0,25.000000,...,163.60,77.27,1,0.0,1.0,-69.14,5,0,0,2
3,78.0,100.0,36.10,103.50,58.0,43.00,12.0,34.0,-3.0,25.000000,...,165.40,77.27,1,0.0,1.0,-69.14,6,0,0,3
4,74.0,100.0,36.00,128.75,69.5,44.50,12.5,34.0,-3.0,25.000000,...,167.20,77.27,1,0.0,1.0,-69.14,7,0,0,4
5,76.0,100.0,36.20,124.00,65.0,41.50,14.0,34.0,0.0,25.000000,...,169.00,77.27,1,0.0,1.0,-69.14,8,0,0,5
6,70.0,100.0,36.10,120.25,65.0,43.00,14.0,34.0,0.0,25.000000,...,165.25,77.27,1,0.0,1.0,-69.14,9,0,0,6
7,67.0,100.0,36.40,130.50,73.0,49.00,14.0,34.0,0.0,24.666667,...,161.50,77.27,1,0.0,1.0,-69.14,10,0,0,7


In [13]:
pd.set_option('display.max_columns',43)
train_data_df.head(8)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,Patient_id,time
0,80.0,100.0,36.50,121.00,58.0,41.00,13.5,34.0,1.0,25.000000,1.0,7.40,47.000000,77.0,42.0,53.000000,49.0,8.0,111.000000,2.10,15.9,107.000000,2.400000,2.8,2.2,3.700000,0.6,1.7,27.900000,8.800000,38.100000,9.900000,223.000000,160.00,77.27,1,0.0,1.0,-69.14,3,0,0,0
1,76.0,100.0,36.25,113.25,61.0,41.50,12.0,34.0,1.0,25.000000,0.5,7.40,45.333333,78.7,42.0,53.000000,49.0,8.0,111.000000,2.08,15.9,108.833333,2.180000,2.8,2.2,3.866667,0.6,1.7,27.900000,8.800000,37.166667,9.900000,223.003759,161.80,77.27,1,0.0,1.0,-69.14,4,0,0,1
2,80.0,100.0,36.25,132.75,71.5,46.25,12.0,34.0,-1.0,25.000000,0.5,7.37,43.666667,80.4,42.0,51.000000,49.0,8.0,110.500000,2.06,15.9,110.666667,1.960000,2.8,2.2,4.033333,0.6,1.7,28.050000,8.800000,36.233333,10.425000,223.007519,163.60,77.27,1,0.0,1.0,-69.14,5,0,0,2
3,78.0,100.0,36.10,103.50,58.0,43.00,12.0,34.0,-3.0,25.000000,0.5,7.34,42.000000,82.1,42.0,49.000000,49.0,8.0,110.000000,2.04,15.9,112.500000,1.740000,2.8,2.2,4.200000,0.6,1.7,28.200000,8.800000,35.300000,10.950000,223.011278,165.40,77.27,1,0.0,1.0,-69.14,6,0,0,3
4,74.0,100.0,36.00,128.75,69.5,44.50,12.5,34.0,-3.0,25.000000,0.5,7.34,45.000000,83.8,42.0,47.000000,49.0,8.0,109.500000,2.02,15.9,132.000000,1.520000,2.8,2.2,4.450000,0.6,1.7,28.350000,8.800000,34.366667,11.475000,223.015038,167.20,77.27,1,0.0,1.0,-69.14,7,0,0,4
5,76.0,100.0,36.20,124.00,65.0,41.50,14.0,34.0,0.0,25.000000,0.5,7.34,48.000000,85.5,42.0,45.000000,49.0,8.0,109.000000,2.00,15.9,151.500000,1.300000,2.8,2.2,4.700000,0.6,1.7,28.500000,8.800000,33.433333,12.000000,223.018797,169.00,77.27,1,0.0,1.0,-69.14,8,0,0,5
6,70.0,100.0,36.10,120.25,65.0,43.00,14.0,34.0,0.0,25.000000,0.5,7.34,44.500000,87.2,42.0,45.000000,49.0,8.0,109.000000,2.00,15.9,139.250000,1.300000,2.8,2.2,4.700000,0.6,1.7,28.500000,8.800000,32.500000,12.000000,223.022556,165.25,77.27,1,0.0,1.0,-69.14,9,0,0,6
7,67.0,100.0,36.40,130.50,73.0,49.00,14.0,34.0,0.0,24.666667,0.5,7.38,41.000000,88.9,42.0,44.333333,49.0,8.0,108.666667,2.00,15.9,127.000000,1.276923,2.8,2.2,4.833333,0.6,1.7,28.966667,9.033333,31.566667,11.366667,223.026316,161.50,77.27,1,0.0,1.0,-69.14,10,0,0,7


In [20]:
train_data_df.shape

(1552210, 43)

In [3]:
len(train_data_df['Patient_id'].unique())

30336

# sort by length

In [4]:
# sort by time length
time_length_series = train_data_df.groupby(['Patient_id'])['SepsisLabel'].count()
time_length_series=time_length_series.sort_values() # sort
time_length_df=time_length_series[:-1].reset_index() # remove total count
time_length_df.columns = ['Patient_id', 'time_length']
time_length_df.head()

,Patient_id,time_length
0,23382,8
1,22552,8
2,15577,8
3,29904,8
4,22554,8


In [5]:
id=np.array(time_length_df['Patient_id'])

In [6]:
id

array([23382, 22552, 15577, ...,  7420,  5279, 26957])

In [66]:
label=train_data_df[train_data_df['Patient_id']==26957]['SepsisLabel'].sum()

In [67]:
label

6

In [79]:
sorted_data=[]
sorted_data_label=[]
for i in id:
    data=train_data_df[train_data_df['Patient_id']==i].iloc[:,:-7].values.tolist()
    if(train_data_df[train_data_df['Patient_id']==i]['SepsisLabel'].sum()>0):
        label=1
    else:
        label=0
    sorted_data.append(data)
    sorted_data_label.append(label)

In [80]:
len(sorted_data)

30335

In [81]:
len(sorted_data_label)

30335

# sort by label

In [87]:
sepsis_data=[]
non_sepsis_data=[]
for i in range(len(sorted_data)):
    if(sorted_data_label[i]==1):
        sepsis_data.append(sorted_data[i])
    else:
        non_sepsis_data.append(sorted_data[i])

In [88]:
len(sepsis_data)

2358

In [89]:
len(non_sepsis_data)

27977

In [90]:
sepsis_data.extend(non_sepsis_data)

In [91]:
len(sepsis_data)

30335

In [116]:
# train
pickle.dump(sepsis_data, open( '../ranking_data/data' + '.seqs', 'wb'), -1) 
# test 200sepsis+400non-sepsis
pickle.dump(test_data, open( '../ranking_data/test_data' + '.seqs', 'wb'), -1) 

NameError: name 'sepsis_data' is not defined

In [2]:
path_string = '../ranking_data/data.seqs'
with open(path_string,'rb') as f:
    a = pickle.load(f)

In [37]:
np.array(a[0]).shape[1]

36

In [907]:
len(data[25])

IndexError: list index out of range

In [763]:
#6 early
pickle.dump(sixhour_early_test_data, open( '../ranking_data/6h_early_test_data' + '.seqs', 'wb'), -1) 

In [923]:
pickle.dump(haha, open( '../ranking_data/6h_early_test_data_label' + '.seqs', 'wb'), -1) 

6h early test data
0：50个，长度 8-6=2，   sepsis 
1:   20个，长度 20-6=14，sepsis
2:  15个， 长度 30-6=24，sepsis
3:  15个， 长度 40-6=34，sepsis
4:  15个， 长度 45-6=39，sepsis
5:  15个， 长度 50-6=44，sepsis 
6:  15个， 长度 51-6=45，sepsis
7:  10个， 长度 55-6=49，sepsis
8:  10个， 长度 61-6=55，sepsis
9:  10个， 长度 65-6=59，sepsis
10:  10个，长度 73-6=67，sepsis
11:  10个，长度 81-6=75，sepsis
12:  5个，长度 90-6=84，sepsis
13:  5个，长度 118-6=112，sepsis
14:  100个，长度 8-6=2，sepsis
15:  40个，长度 15-6=9，sepsis
16:  40个，长度 9-6=3，non-sepsis
17:  40个，长度 11-6=5，non-sepsis
18:  300个,长度 20-6=14，non-sepsis
19:  200个,长度 32-6=26，non-sepsis
20:  200个,长度 36-6=30，non-sepsis
21:  200个,长度 43-6=37，non-sepsis
22:  200个,长度 48-6=42，non-sepsis
23:  100个,长度 55-6=49，non-sepsis
24:  100个,长度 59-6=53，non-sepsis

In [7]:
path_string = '../ranking_data/6h_early_test_data.seqs'
with open(path_string,'rb') as f:
    data = pickle.load(f)
path_string = '../ranking_data/6h_early_test_data_label.seqs'
with open(path_string,'rb') as f:
    label = pickle.load(f)

In [14]:
a[0][7][19]

0.7957894736842106

In [ ]:
a[0][7][19]